# Exercise - Data<a id="sec_overview"></a>

The aim of this exercise is to strengthen your understanding of data structures and their optimization and visualization. Below you \
can see an example of an Entity-Relationship Diagram (ERD). The model represents the storage of information about movies. In the lecture \
you already heard of entities, attributes and their releationsships. With in the example we can identify the parts as: 

- Entities: Movie, Person, Country
- Attributes: name, first_name, last_name, population, duration, date_of_birth, area e.g.
- Relationships: the lines in between the entities, symbols representing one-to-one, one-to-many (Person → Movie) or many-to-many (Movie ↔ Country)

The diagram is an overview of how the stored data is structured and can be used to visualize redundancies.\
NOTE: The rows from which a line leads to another entity represent a relationship and actually contain a key to the referenced entity. Strictly speaking, they are NOT attributes. (for this ERD: directed_by and produced_in)

![Image Example ERD](example_erd.png)

<div class="alert alert-block alert-success">
This is an exercise in understanding the relationships between entities and their attributes based on Chapter 4 “Data”.

## Content <a id="sec_toc"> </a> 

[Database Visualization with ERDs](#sec_a)

[Adding an Entity to the model](#sec_b)

[Creating Instances for the given model](#sec_c)

[Creating Table-Based-Representation of Instances](#sec_d)

[Compare memory requirements](#sec_e)

In [ ]:
! mamba install --channel conda-forge pygraphviz -y
! pip install erdantic pandas pympler pydantic

Import the needed libaries:

In [6]:
import erdantic as erd
from enum import Enum

import erdantic.examples.pydantic
from pydantic import BaseModel
from typing import List, Optional

import pandas as pd 
from pympler import asizeof
import random as r
import sqlite3
import os

## Database Visualization with ERDs<a id="sec_a"></a> 

To create the ERD, the entities "Department," "Faculty," "Lecture," and "Student" are defined as classes \
in the following code snippet. The attributes of each entity are specified within the respective class, \
along with their required variable types. Attributes that reference another entity represent the \
relationships between these entities.

In [ ]:
# HAVE A LOOK BUT DO NOT CHANGE: Overview over entities and their attributes and relationships
class Role(str, Enum): # This class defines the possible positions of a faculty, it is not an Entity!!!!
    PROFESSOR = "professor"
    SENIOR_SCIENTIST = "senior_scientist"
    LAB_TECHNICIAN = "lab_technician"
    STUDENT_ASSISTANT = "student_assistant"

class Department(BaseModel):
    """ 
    Attributes: 
        name (str): Name of department
        dep_number (int): Identification number of department
    """
    name: str
    dep_number: int
    
class Faculty(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the faculty (member of the university)
        personnel_number (int): Identification Number of the university member (Personalnummer)
        role (Role): Position the university member holds (Avaliable are: "professor", "senior_scientist","lab_technician", "student_assistant")

    Relationships: 
        member_of (Department): Department where the faculty member is employeed 
    """
    name: str
    personnel_number: int 
    role: Role
    member_of: Department

class Student(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the student 
        matriculation_number (int): Identification Number of the student (Matrikelnummer)
        major (str): The student's field of study
    """
    name: str
    matriculation_number: int
    major: str

class Lecture(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the lecture
        lecture_number (int): Identification Number of the lecture
        
    Relationships: 
        held_by (Faculty): Faculty that holds the lecture 
        offered_by (Department): Department, which offers the lecture 
        attendees (List of Student): Students that are attending the lecture
    """
    name: str
    lecture_number: int
    held_by: Faculty
    offered_by: Department
    attendees: List[Student]


In [ ]:
# DO NOT CHANGE: Creating the ERD starting from the entity "Lecture"
erd1 = erd.create(Lecture)
display(erd1)

[Back](#sec_toc)

## Adding an Entity to the model <a id="sec_b"></a> 

As several students will inscribe into the same field of study it makes sense to create a new entity "Major" that summerizes \
the information about one study program. The entity "Student" must be redefined such that the variable type for major is no \
longer a string but represents the relationship to the new entity "Major". The "Faculty" entity also has a new attribute \
“department_head”, which gives if the faculty member is the department head.

In [ ]:
# HAVE A LOOK BUT DO NOT CHANGE:
class Degree(str, Enum): # This class devines the possible type of degrees for the major, this is NOT an entity!!
    BACHELOR = "BSc"
    MASTER = "MSc"
    DOCTOR = "Dr"

#  New entity “Major”, which specifies the student's field of study
class Major(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the major
        major_number (int): Identification Number of the major 
        degree_type (str): The type of degree that can be achieved through the major (Avaliable are: "BSc", "MSc", "Dr")
    """
    name: str
    major_number: int
    degree_type: Degree

# Redefine Student so that you can add the new entity Major
class Student(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the student 
        matriculation_number (int): Identification Number of the student (Matrikelnummer)
    
    Relationships:
        studies (Major): The student's field of study
    """
    name: str
    matriculation_number: int
    studies: Major

# Adding a new attribute to the Faculty entity
class Faculty(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the faculty (member of the university)
        personnel_number (int): Identification Number of the university member (Personalnummer)
        role (Role): Position the university member holds (Avaliable are: "professor", "senior_scientist","lab_technician", "student_assistant")
        department_head (bool): Gives if the member of the university is the head of the department where they are employeed
    
    Relationships:
        member_of (Department): Department where the faculty member is employeed
    """
    name: str
    personnel_number: int 
    role: Role
    member_of: Department
    department_head: bool

# Redefine Lecture as the relationsship to the Faculty and Student entities must be updated
class Lecture(BaseModel): 
    """ 
    Attributes: 
        name (str): Name of the lecture
        lecture_number (int): Identification Number of the lecture
    
    Relationships:
        held_by (Faculty): Faculty that holds the lecture 
        offered_by (Department): Department, which offers the lecture 
        attendees (List of Student): Students that are attending the lecture
    """
    name: str
    lecture_number: int
    held_by: Faculty
    offered_by: Department
    attendees: List[Student]

In [ ]:
# DO NOT CHANGE: Create the new ERD
erd2 = erd.create(Lecture)
display(erd2)

[Back](#sec_toc)

## Creating Instances for the given model  <a id="sec_c"></a>  

In [ ]:
# DO NOT CHANGE: Method that creates a easy-to-read overview 
def display_chair_info(department, lectures, faculty_members):
    result = f"{department.name} (Nr.: {department.dep_number})\n"
    result += "-" * 33 + "\n"
    
    # Format Lectures
    result += "Offered Lectures:\n"
    department_lectures = [lecture for lecture in lectures if lecture.offered_by == department]
    
    for idx, lecture in enumerate(department_lectures, start=1):
        result += f"{idx}. {lecture.name} (Lecture Number: {lecture.lecture_number})\n"
        result += f"   - Lecturer: {lecture.held_by.name}\n"
        result += f"     - Personnel Number: {lecture.held_by.personnel_number}\n"
        result += f"     - Role: {lecture.held_by.role}\n"
        result += f"     - Department Head: {'Yes' if lecture.held_by.department_head else 'No'}\n"
        
        # Attendees
        if lecture.attendees:
            result += "   - Attendees:\n"
            for student_idx, student in enumerate(lecture.attendees, start=1):
                result += f"     {student_idx}. {student.name}\n"
                result += f"        - Matriculation Number: {student.matriculation_number}\n"
                result += f"        - Major: {student.studies.name} ({student.studies.major_number})\n"
                result += f"          - Degree Type: {student.studies.degree_type}\n"
        else:
            result += "   - Attendees: None\n"
    
    # Faculty Team
    result += "\nTeam:\n"
    department_faculty = [member for member in faculty_members if member.member_of == department]
    
    for idx, member in enumerate(department_faculty, start=1):
        result += f"{idx}. {member.name}\n"
        result += f"   - Personnel Number: {member.personnel_number}\n"
        result += f"   - Role: {member.role}\n"
        result += f"   - Department Head: {'Yes' if member.department_head else 'No'}\n"
    
    return result

TASK: Fill in the empty attributes/relationships of the given instances in the following code with permitted values.\
Below an overview of the created Department is given.

In [ ]:
# Fill out the attributes/relationships where you only see dots (....)

department = { # Instance of Department 
    "name":"Cyber-Physical-Systems", # DO NOT CHANGE THIS ATTRIBUTE
    "dep_number": 5, # DO NOT CHANGE THIS ATTRIBUTE
}
# Initialize the instance of Department as a python object with attributes/relationships given above:
dep1 = Department(**department)

department_head = { # Instance of Faculty that is the head of the department
    "name":"Elmar Rueckert", # DO NOT CHANGE THIS ATTRIBUTE
    "personnel_number":12345678, # DO NOT CHANGE THIS ATTRIBUTE
    "role": "professor", # Avaliable roles are: "professor", "senior_scientist","lab_technician", "student_assistant"
    "member_of": dep1, # DO NOT CHANGE THIS ATTRIBUTE
    "department_head": True # DO NOT CHANGE THIS ATTRIBUTE
}
department_member = { # Instance of Faculty which is a teammeber of the department
    "name":......, 
    "personnel_number": ......, 
    "role": ......, # Avaliable roles are: "professor", "senior_scientist","lab_technician", "student_assistant"
    "member_of": dep1, # DO NOT CHANGE THIS ATTRIBUTE
    "department_head": False # DO NOT CHANGE THIS ATTRIBUTE
}

# Initialize the instances of Department as two python objects with attributes/relationships given above:
f1 = Faculty(**department_head) 
f2 = Faculty(**department_member)
faculty = [f1,f2]

major_one = { # Instance of Major (one)
    "name":"Industrial Data Science", # DO NOT CHANGE THIS ATTRIBUTE
    "major_number":530, # DO NOT CHANGE THIS ATTRIBUTE
    "degree_type": "BSc" # Avaliable degrees are: "BSc", "MSc", "Dr",  DO NOT CHANGE THIS ATTRIBUTE
}
major_two = { # Instance of Major (two)
    "name":......, 
    "major_number":....., 
    "degree_type": ...... # Avaliable degrees are: "BSc", "MSc", "Dr"
}
# Initialize the instances of Major as two python objects with attributes/relationships given above:
m1 = Major(**major_one) 
m2 = Major(**major_two)
majors = [m1,m2]

student_one = { # Instance of Student (one)
    "name":"Hansi Hoelzel", # DO NOT CHANGE THIS ATTRIBUTE
    "matriculation_number":19021957, # DO NOT CHANGE THIS ATTRIBUTE
    "studies": m1 # DO NOT CHANGE THIS RELATIONSHIP
}
student_two = { # Instance of Student (two)
    "name":......, 
    "matriculation_number":......, 
    "studies": ..... # Avaliabe Major instances: m1, m2
}
# Initialize the instances of Student as two python objects with attributes/relationships given above:
s1 = Student(**student_one)
s2 = Student(**student_two)
students = [s1,s2]

lecture_one = { # Instance of Lecture which is inscribed by the first Student instance
    "name": "Data Modeling", # DO NOT CHANGE THIS ATTRIBUTE
    "lecture_number": 190021, # DO NOT CHANGE THIS ATTRIBUTE
    "held_by": f1, # DO NOT CHANGE THIS RELATIONSHIP
    "offered_by": dep1, # DO NOT CHANGE THIS RELATIONSHIP
    "attendees": [s1] # DO NOT CHANGE THIS RELATIONSHIP
}
lecture_two = { # Instance of Lecture which is inscribed by the first and second Student instance
    "name": ......,
    "lecture_number": ......,
    "held_by": ......, # Avaliable Faculty instances: f1, f2
    "offered_by": dep1, # DO NOT CHANGE THIS RELATIONSHIP
    "attendees": [s1,s2] # DO NOT CHANGE THIS RELATIONSHIP
}

lecture_three = { # Instance of Lecture which is not inscribed yet
    "name": ......,
    "lecture_number": ......,
    "held_by": ......, # Avaliable Faculty instances: f1, f2
    "offered_by": dep1, # DO NOT CHANGE THIS RELATIONSHIP
    "attendees": [] # DO NOT CHANGE THIS RELATIONSHIP
}

# Initialize the instances of Lecture as three python objects with attributes/relationships given above:
l1 = Lecture(**lecture_one)
l2 = Lecture(**lecture_two)
l3 = Lecture(**lecture_three)
lectures = [l1,l2,l3]

# Print infos about created department
print("Overview of the created Department:")
print(display_chair_info(dep1, lectures, faculty))

TASK: Again fill in the empty attributes of the given instances in the following code with permitted values.

In [ ]:
# Creates two new instances of department
department_two = {
    "name": ...... , 
    "dep_number": ......,
}
dep2 = Department(**department_two)

department_three = {
    "name":......, 
    "dep_number": ......,
}
# Initialize the instances of Department as three python objects with attributes/relationships given above:
dep3 = Department(**department_three)
departments= [dep1,dep2,dep3]

student_three = { # Instance of Student (three) that is not inscribed in any lectures
    "name":......., 
    "matriculation_number": ......., 
    "studies": .......
}
s3 = Student(**student_three)
students = [s1,s2,s3]

major_three = { # Instance of Major (three) that nobody has chosen
    "name":......, 
    "major_number": ......, 
    "degree_type": ......
}
m3 = Major(**major_three)
majors = [m1,m2,m3]

lecture_four = { # Instance of Lecture which is part of the new department
    "name": ......,
    "lecture_number": ......,
    "held_by": .......,
    "offered_by": dep2, # DO NOT CHANGE THIS RELATIONSHIP
    "attendees": [] # DO NOT CHANGE THIS RELATIONSHIP
}
l4 = Lecture(**lecture_four)
lectures = [l1,l2,l3,l4]

faculty_member = { # Instance  of Faculty which is a member of the new department
    "name":......., 
    "personnel_number": ......., 
    "role": ......, # Avaliable roles are: "professor", "senior_scientist","lab_technician", "student_assistant"
    "member_of": dep2, # DO NOT CHANGE THIS RELATIONSHIP
    "department_head": ......
}
f3 = Faculty(**faculty_member)
faculty = [f1,f2, f3]


[Back](#sec_toc)

## Creating Table-Based-Representation of Instances <a id="sec_d"></a>

The following code snippet generates a complete table based on the information from the instances created [above](#sec_c). 

This table includes only complete entries! This means it adheres to the following criteria:

- Students: Only students who are enrolled in offered lectures are displayed.
- Lectures: Only lectures that have at least one enrolled student are included.
- Faculty: Only faculty members teaching one of these lectures are listed.
- Majors: Only majors that are actively taken by the listed students are shown.

In [ ]:
# DO NOT CHANGE: Creating full information table
# Create DataFrame for full row informations, define columns and the type of variable that is expected
df_full = pd.DataFrame({
    'Student_Name':  pd.Series(dtype='string'),
    'Matriculation_Number':pd.Series(dtype='int'),
    'Student_Major': pd.Series(dtype='string'),
    'Student_Major_ID': pd.Series(dtype='int'),
    'Degree_Type':pd.Series(dtype='object'),
    'Lecture':  pd.Series(dtype='string'),
    'Lecture_Nr': pd.Series(dtype='int'),
    'Lecture_Department': pd.Series(dtype='string'),
    'Lecture_Department_ID': pd.Series(dtype='int'),
    'Faculty': pd.Series(dtype='string'),
    'Faculty_PNr':pd.Series(dtype='int'),
    'Faculty_Role':pd.Series(dtype='object'),
    'Faculty_Department': pd.Series(dtype ='string'), 
    'Faculty_Department_ID': pd.Series(dtype ='string'), 
    'Faculty_is_Departmenthead':pd.Series(dtype='bool'),
    })

# Including information about the all instances created above depending on the lecture entity
for l in lectures:
    for student in l.attendees:
        df_Student1 = pd.DataFrame({
            'Student_Name': [student.name],
            'Matriculation_Number':[student.matriculation_number],
            'Student_Major':[student.studies.name],
            'Student_Major_ID': [student.studies.major_number],
            'Degree_Type':[student.studies.degree_type],
            'Lecture': [l.name],
            'Lecture_Nr': [l.lecture_number],
            'Faculty': [l.held_by.name],
            'Faculty_PNr':[l.held_by.personnel_number],
            'Faculty_Role':[l.held_by.role],
            'Faculty_Department': [l.held_by.member_of.name],
            'Faculty_Department_ID': [l.held_by.member_of.dep_number],
            'Faculty_is_Departmenthead':[l.held_by.department_head],
            'Lecture_Department':[l.offered_by.name],
            'Lecture_Department_ID':[dep1.dep_number]
        })
        df_full = pd.concat([df_full, df_Student1],ignore_index=True)

df_full

In order to be able to use all of the above information, it makes sense to create tables for each entity \
individually, as otherwise information would be lost. The following code snippets are used to create tables \
for each entity.

In [ ]:
# DO NOT CHANGE: Creating table for entity "Department"
# Create Department DataFrame and define the type of variable that is expected
df_department =  pd.DataFrame({
    'ID': pd.Series(dtype='int'),
    'Name':  pd.Series(dtype='string'),
    })

# Method that adds departments to DataFrame
def add_department(df, dep):
    new_department = pd.DataFrame({
        'ID': [dep1.dep_number],
        'Name': [dep1.name],
    })
    df = pd.concat([df,new_department], ignore_index=True)
    return df

# Add lectures of the instances that are defined above
for d in departments:
    df_department = add_department(df_department, d)
df_department.set_index('ID',inplace=True )
print("Table for entity Department:")
df_department

In [ ]:
# DO NOT CHANGE: Creating table for entity "Major"
# Create Major DataFrame and define the type of variable that is expected
df_major = pd.DataFrame({
    'ID': pd.Series(dtype='int'),
    'Name': pd.Series(dtype='string'),
    'Degree_Type':pd.Series(dtype='object')
    })

# Method that adds majors to DataFrame
def add_major(df, major):
    new_major = pd.DataFrame({
        'Name': [major.name],
        'ID': [major.major_number],
        'Degree_Type':[major.degree_type]
    })
    df = pd.concat([df,new_major], ignore_index=True)
    return df

# Add majors of the instances that are defined above
for m in majors:
    df_major = add_major(df_major, m)
df_major.set_index('ID', inplace=True )
print("Table for entity Major:")
df_major

In [ ]:
# DO NOT CHANGE: Creating table for entity "Student"
# Create Student DataFrame and define the type of variable that is expected
df_student = pd.DataFrame({
    'Name': pd.Series(dtype='string'),
    'Matriculation_Number':pd.Series(dtype='int'),
    'Major_ID': pd.Series(dtype='int')
    })

# Method that adds students to DataFrame
def add_student(df, student):
    new_student = pd.DataFrame({
        'Name': [student.name],
        'Matriculation_Number': [student.matriculation_number],
        'Major_ID': [student.studies.major_number]
    })
    df = pd.concat([df,new_student], ignore_index=True)
    return df

# Add students of the instances that are defined above
for s in students:
    df_student = add_student(df_student,s)
df_student.set_index('Matriculation_Number',inplace=True )
print("Table for entity Student:")
df_student

In [ ]:
# DO NOT CHANGE: Creating table for entity "Faculty"
# Create Faculty DataFrame and define the type of variable that is expected
df_faculty = pd.DataFrame({
    'Personnel_Number':pd.Series(dtype='int'),
    'Name': pd.Series(dtype='string'),
    'Role':pd.Series(dtype='object'),
    'Department_ID': pd.Series(dtype='int'),
    'Is_Departmenthead':pd.Series(dtype='bool'),
    })

# Method that adds faculties to DataFrame
def add_faculty(df, faculty):
    new_faculty = pd.DataFrame({
        'Personnel_Number':[faculty.personnel_number],
        'Name': [faculty.name],
        'Role':[faculty.role],
        'Department_ID': [faculty.member_of.dep_number],
        'Is_Departmenthead':[faculty.department_head],
    })
    df = pd.concat([df,new_faculty], ignore_index=True)
    return df

# Add faculties of the instances that are defined above
for f in faculty: 
    df_faculty = add_faculty(df_faculty, f)
df_faculty.set_index('Personnel_Number',inplace=True )
print("Table for entity Faculty:")
df_faculty

In [ ]:
# DO NOT CHANGE: Creating table for entity "Lecture"
# Create Lecture DataFrame and define the type of variable that is expected
df_lecture = pd.DataFrame({
    'ID': pd.Series(dtype='int'),
    'Name':  pd.Series(dtype='string'),
    'Lecturer_PNr':pd.Series(dtype='int'),
    'Department_ID':pd.Series(dtype="int")
    })

# Method that adds lectures to DataFrame
def add_lecture(df, lecture):
    new_lecture = pd.DataFrame({
        'ID': [lecture.lecture_number],
        'Name': [lecture.name],
        'Lecturer_PNr':[lecture.held_by.personnel_number],
        'Department_ID':[lecture.offered_by.dep_number]
    })
    df = pd.concat([df,new_lecture], ignore_index=True)
    return df

# Add lectures of the instances that are defined above
df_lecture = add_lecture(df_lecture, l1)
df_lecture = add_lecture(df_lecture, l2)
df_lecture = add_lecture(df_lecture, l3)
df_lecture = add_lecture(df_lecture, l4)
df_lecture.set_index('ID',inplace=True )
print("Table for entity Lecture:")
df_lecture

Within the tables, only one-to-many relationships are represented, as the identification number (primary key)\
can be included directly in the table on the "many" side. (Example: Each movie in a table can directly reference\
its director, as a movie can have only one director.) Relationships between entities that are not already \
represented in the tables (many-to-many relationships) require a separate table to store the necessary information. \
The following code snippets create this table.

In [ ]:
# DO NOT CHANGE: Creating table for relationship "Lecture" <-> "Student"
# Create Relationship DataFrame for Lecture to Student
df_lecture_attendee =  pd.DataFrame({
    'Lecture_ID': pd.Series(dtype='int'),
    'Attendee_ID':  pd.Series(dtype='int'),
    })

# Method that adds relationships to DataFrame based on the given lecture
def add_relation_lecture_student (df,lecture):
    for student in lecture.attendees:
        new_relation = pd.DataFrame ({
            'Lecture_ID': [lecture.lecture_number],
            'Attendee_ID':  [student.matriculation_number],
        })
        df = pd.concat([df,new_relation], ignore_index = True)
    return df

# Add relationships of the instances that are defined above 
df_lecture_attendee = add_relation_lecture_student(df_lecture_attendee,l1)
df_lecture_attendee = add_relation_lecture_student(df_lecture_attendee,l2)
df_lecture_attendee = add_relation_lecture_student(df_lecture_attendee,l3)
df_lecture_attendee = add_relation_lecture_student(df_lecture_attendee,l4)
print("Table for relationship Lecture to Student (Attendee):")
df_lecture_attendee

[Back](#sec_toc)

## Compare memory requirements <a id="sec_e"></a> 

As we have worked above with python objects and tables, it would be interesting to compare\
which of these needs more memory.The following code snipped compares the required memory \
of python objects with their table-based representations in a Database (SQL). To make the \
comparison more simple, a new entity "Book" is created. From these entities the code randomly \
creates n instances and stores the information in a database as well. 

TASK: Try different numbers of instances to compare the memory

In [ ]:
# Define number of created instances
n = None # CHANGE the number of instances from None to a number in the range [1,1500]

In [ ]:
# DO NOT CHANGE: Creating 1000 instances of "Book" entity and calculate memory requirements
# Defining the new entity "Book"
class Book(BaseModel):
    """
    Attributes: 
        isbn (int): Represents the International Standard Book Number
        author (str): Name of person who has written the book
        title (str): Title of the book
    """
    isbn: int
    author: str
    title: str

library = []

if (n == None):
    print("The number of instances is not defined. Please change the value from None to a valide value.")
elif ( n> 1500):
    print("The number of instances is out of range. Please enter a number in the range of 1 to 1500.")
else: 
    # Creating n random book instances 
    library = [Book(isbn=100000000+i, title= f"Storybook_{i}", author=f"Author_{r.randint(1,20)}") for i in range(n)]

# Creating database for books
db_path = "library.db"
# delete database if it already exists
if os.path.exists(db_path):
    try: os.remove(db_path)
    except PermissionError: 
        print("Error occured please restart the kernel")   
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
# delete table if it already exists
cursor.execute("DROP TABLE IF EXISTS library")
# create new table in database
cursor.execute("CREATE TABLE IF NOT EXISTS library (isbn INTEGER, author TEXT, title TEXT)")
# Adding all the books to the database
cursor.executemany("INSERT INTO library (isbn, author, title) VALUES (?, ?, ?)",
    [(book.isbn, book.author, book.title) for book in library])
conn.commit()
conn.close() # close connection to database

# Memory requirement of "Book" instances
memory_objects = 0
for obj in library:
    memory_objects += asizeof.asizeof(obj)

# Memory requirement table-based representation (SQL-Database)
memory_database = os.path.getsize(db_path)

if(n!=None):
    if(n<=1500 and n>=1):
        print(f"Number of books in library: {len(library)}")
        print("Memory requirements:")
        print(f"Python Objects: {memory_objects} Bytes")
        print(f"Table-Based representation (SQL): {memory_database} Bytes")

The number of instances is out of range. Please enter a number in the range of 1 to 1500.
Error occured please restart the kernel


[Back](#sec_toc)